# Cronbach’s Alpha

In [3]:
import pingouin as pg
from pingouin import cronbach_alpha

In [4]:
file_path = "movie_psychology_cleanedData.xls"
data = pd.read_excel(file_path)

In [5]:
print(data.head())

   user_id  E1  E2  E3  E4  E5  E6  A1  A2  A3  ...  N2  N3  N4  O1  O2  O3  \
0      264   1   1   1   1   1   1   1   1   1  ...   1   1   1   1   1   1   
1      262   4   4   4   2   4   4   4   2   4  ...   4   4   4   4   4   4   
2      261   1   2   2   3   3   2   5   2   4  ...   2   4   3   2   3   4   
3      260   1   4   3   3   3   3   3   3   3  ...   3   3   3   4   2   3   
4      259   4   4   2   4   2   2   4   4   2  ...   4   3   3   2   3   4   

   O4  gender  Movie_watching_frequency  \
0   1    male      several times a week   
1   3    male      several times a week   
2   3  female      At least once a week   
3   3    male    Less than once a month   
4   3  female    Less than once a month   

                                      favorite_genre  
0  comedy, romance,action,horror,mystery,animatio...  
1                               comedy,action,sci-fi  
2          comedy, romance,horror,mystery,historical  
3                                             

In [6]:
dimensions = {
    "Extraversion": ["E1", "E2", "E3", "E4", "E5", "E6"],
    "Agreeableness": ["A1", "A2", "A3", "A4"],
    "Conscientiousness": ["C1", "C2", "C3", "C4", "C5"],
    "Neuroticism": ["N1", "N2", "N3", "N4"],  
    "Openness": ["O1", "O2", "O3", "O4"]
}


In [7]:
results = {}
for dimension, columns in dimensions.items():
    subset = data[columns]
    alpha, _ = cronbach_alpha(subset)
    results[dimension] = alpha

In [8]:
for dimension, alpha in results.items():
    print(f"{dimension}: Cronbach's Alpha = {alpha:.2f}")

Extraversion: Cronbach's Alpha = 0.84
Agreeableness: Cronbach's Alpha = 0.40
Conscientiousness: Cronbach's Alpha = 0.71
Neuroticism: Cronbach's Alpha = 0.80
Openness: Cronbach's Alpha = 0.56


#  McDonald’s Omega

In [11]:
from semopy import ModelMeans

In [14]:
from semopy import Model

In [16]:
# 定义函数：计算 McDonald's Omega
def calculate_omega(dim_name, questions):
    # 构造单因子模型描述
    model_desc = f"{dim_name} =~ " + " + ".join(questions)
    
    # 初始化模型
    model = Model(model_desc)
    
    try:
        # 拟合模型
        result = model.fit(data[questions])
        
        # 提取因子负荷量和误差方差
        params = model.inspect()  # 提取所有参数
        loadings = params.loc[params['op'] == '~', 'Estimate']  # 因子负荷量
        error_var = params.loc[params['op'] == '~~', 'Estimate']  # 误差方差
        
        # 计算 Omega
        lambda_sum = loadings.sum()
        psi_sum = error_var.sum()
        omega = (lambda_sum ** 2) / ((lambda_sum ** 2) + psi_sum)
        
        return omega
    except Exception as e:
        # 捕捉可能的错误（如数据问题或维度问题）
        print(f"Error calculating Omega for {dim_name}: {e}")
        return None

# 对每个维度进行 McDonald's Omega 检测
omega_results = {}
for dim_name, questions in dimensions.items():
    omega_results[dim_name] = calculate_omega(dim_name, questions)

# 输出结果
for dim, omega_value in omega_results.items():
    if omega_value is not None:
        print(f"{dim}: McDonald's Omega = {omega_value:.2f}")
    else:
        print(f"{dim}: Unable to calculate Omega.")

Extraversion: McDonald's Omega = 0.91
Agreeableness: McDonald's Omega = 1.00
Conscientiousness: McDonald's Omega = 0.97
Neuroticism: McDonald's Omega = 0.91
Openness: McDonald's Omega = 0.89


# Correlation Matrix

In [17]:
correlation_matrix = data[dimensions["Openness"]].corr()
print("Openness Correlation Matrix:")
print(correlation_matrix)

Openness Correlation Matrix:
          O1        O2        O3        O4
O1  1.000000  0.185800  0.267963  0.142308
O2  0.185800  1.000000  0.261464  0.335550
O3  0.267963  0.261464  1.000000  0.260953
O4  0.142308  0.335550  0.260953  1.000000


In [18]:
correlation_matrix = data[dimensions["Agreeableness"]].corr()
print("Agreeableness Correlation Matrix:")
print(correlation_matrix)

Agreeableness Correlation Matrix:
          A1        A2        A3        A4
A1  1.000000  0.107468  0.074938  0.027389
A2  0.107468  1.000000  0.178421  0.051434
A3  0.074938  0.178421  1.000000  0.376487
A4  0.027389  0.051434  0.376487  1.000000


In [2]:
!pip install pingouin

  Obtaining dependency information for pingouin from https://files.pythonhosted.org/packages/eb/56/6d3607f3a78aee1de8e5466f5171722c8e344266a12dc44ccb73d024b3b3/pingouin-0.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for pandas-flavor from https://files.pythonhosted.org/packages/67/1a/bfb5574b215f530c7ac5be684f33d60b299abbebd763c203aa31755f2fb2/pandas_flavor-0.6.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 453.1 kB/s eta 0:00:00a 0:00:01


In [10]:
!pip install semopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 859.9 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for numdifftools from https://files.pythonhosted.org/packages/a3/5c/37cd5db8c465db2664b2219410b8bc7743da6edb1b616b5d13008bd7cac2/numdifftools-0.9.41-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 536.3 kB/s eta 0:00:00a 0:00:01
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659681 sha256=9247e694f923f0515e296f0d150f2dda2d2aa073b796b8477b7f6a7c4e3f0bfc
  Stored in directory: /Users/shufanfan/Library/Caches/pip/wheels/04/ff/18/abad931a60dba4c2a85ac19cea16c06c6eaf887e14f9226045
Successfully built semopy
